In [1]:
from ortools.linear_solver import pywraplp
from instances import *
from pathlib import Path
import numpy as np
from itertools import chain
from time import time_ns
import pandas as pd

In [2]:
shapes = [
  (100,100),
  (150,100),
  (100,150),
  (200,200),
  (250,200),
  (200,250),
  (300,300),
  (350,300),
  (300,350),
  (500,500)
]

In [3]:
create_instances(shapes=shapes, random_state=42)

In [4]:
output_folder = Path() / 'instances'

In [9]:
res = []

## Modelo Original

In [10]:
%%time
for idx,details in enumerate(get_instances()):
  start_time = time_ns()
  s,d,c = details
  ns = len(s)
  nd = len(d)
  x = []
  objective_terms = []

  solver = pywraplp.Solver.CreateSolver('GLOP')

  for i in range(ns):
    x.append([])  
    for j in range(nd):
      x[i].append(solver.NumVar(0, solver.Infinity(), f'x_{i}_{j}'))

  for i in range(ns):
    solver.Add(solver.Sum([x[i][j] for j in range(nd)]) <= s[i])

  for j in range(nd):
    solver.Add(solver.Sum([(x[i][j]) for i in range(ns) ]) >= d[j])

  for i in range(ns):
    for j in range(nd):
      objective_terms.append(c[i][j] * x[i][j])

  solver.Minimize(solver.Sum(objective_terms))
  status = solver.Solve()
  end_time = time_ns()

  if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    res.append([idx, 'original', round(solver.Objective().Value(), 2), round((end_time - start_time) * 1e-9, 2)])

CPU times: user 35.1 s, sys: 128 ms, total: 35.3 s
Wall time: 35.3 s


## Solução do Dual

In [11]:
%%time
for idx,details in enumerate(get_instances()):
  start_time = time_ns()
  s,d,c = details
  ns = len(s)
  nd = len(d)
  y = []
  objective_terms = []

  solver = pywraplp.Solver.CreateSolver('GLOP')

  for i in range(ns + nd):
    y.append(solver.NumVar(0.0, solver.Infinity(), f'y_{i}'))

  for i in range(ns):
    for j in range(nd):
      solver.Add( y[i] - y[j + ns] >= -c[i][j] )

  for i, b in zip(list(range(ns+nd)), chain(s, list(map(lambda x: -x, d)))):
    objective_terms.append(y[i] * b)

  solver.Minimize(solver.Sum(objective_terms))
  status = solver.Solve()
  end_time = time_ns()

  if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    res.append([idx, 'dual', round(-solver.Objective().Value(), 2), round((end_time - start_time) * 1e-9, 2)])

CPU times: user 22.3 s, sys: 11.9 ms, total: 22.3 s
Wall time: 22.3 s


## Original com integralidade

In [12]:
%%time
for idx,details in enumerate(get_instances()):
  start_time = time_ns()
  s,d,c = details
  ns = len(s)
  nd = len(d)
  x = []
  objective_terms = []

  solver = pywraplp.Solver.CreateSolver('GLOP')

  for i in range(ns):
    x.append([])
    for j in range(nd):
      x[i].append(solver.IntVar(0, solver.Infinity(), f'x_{i}_{j}'))

  for i in range(ns):
    solver.Add(solver.Sum([x[i][j] for j in range(nd)]) <= s[i])

  for j in range(nd):
    solver.Add(solver.Sum([(x[i][j]) for i in range(ns) ]) >= d[j])

  for i in range(ns):
    for j in range(nd):
      objective_terms.append(c[i][j] * x[i][j])

  solver.Minimize(solver.Sum(objective_terms))
  status = solver.Solve()
  end_time = time_ns()

  if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    res.append([idx, 'original-int',round(solver.Objective().Value(), 2), round((end_time - start_time) * 1e-9, 2)])

CPU times: user 31.1 s, sys: 116 ms, total: 31.2 s
Wall time: 31.2 s


## Dual com integralidade

In [13]:
%%time
for idx,details in enumerate(get_instances()):
  start_time = time_ns()
  s,d,c = details
  ns = len(s)
  nd = len(d)
  y = []
  objective_terms = []

  solver = pywraplp.Solver.CreateSolver('GLOP')

  for i in range(ns + nd):
    y.append(solver.IntVar(0.0, solver.Infinity(), f'y_{i}'))

  for i in range(ns):
    for j in range(nd):
      solver.Add( y[i] - y[j + ns] >= -c[i][j] )

  for i, b in zip(list(range(ns+nd)), chain(s, list(map(lambda x: -x, d)))):
    objective_terms.append(y[i] * b)

  solver.Minimize(solver.Sum(objective_terms))
  status = solver.Solve()
  end_time = time_ns()

  if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    res.append([idx, 'dual-int', round(-solver.Objective().Value(), 2), round((end_time - start_time) * 1e-9, 2)])

CPU times: user 21.4 s, sys: 32 ms, total: 21.4 s
Wall time: 21.4 s


In [14]:
df_res = pd.DataFrame(res, columns=['instancia', 'modelo', 'valor', 'tempo'])

In [21]:
df_res.instancia = df_res.instancia + 1
df_res

,instancia,modelo,valor,tempo
0,1,original,12250.28,0.43
1,2,original,16690.85,2.02
2,3,original,19678.34,1.36
3,4,original,22892.09,4.95
4,5,original,24855.02,2.14
5,6,original,18964.90,4.76
6,7,original,22919.97,3.93
7,8,original,21890.31,0.45
8,9,original,18029.00,0.26
9,10,original,21164.77,14.73
